## Working with Related Objects

In this section, we will cover one more _essential ORM concept_, which is __how the ORM interacts with mapped classes that refer to other objects__. In the section `Declaring Mapped Classes`, the _mapped class_ examples made use of a construct called `relationship()`. This construct __defines a linkage__ between _two different mapped classes_, or from a _mapped class to itself_, the _latter_ of which is called a __`self-referential relationship`__. To describe the _basic idea_ of `relationship()`, first we'll review the mapping in short form, omitting the _Column mappings_ and other directives.

In [1]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, select
from sqlalchemy.orm import Session, registry, relationship

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
mapped_registry = registry()
Base = mapped_registry.generate_base()
session = Session(engine)

In [3]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [4]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [5]:
mapped_registry.metadata.create_all(engine)

2022-10-10 08:41:28,761 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-10 08:41:28,762 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-10 08:41:28,764 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-10 08:41:28,766 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-10-10 08:41:28,767 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-10 08:41:28,768 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-10 08:41:28,769 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-10 08:41:28,770 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-10-10 08:41:28,771 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-10 08:41:28,773 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-10 08:41:28,774 INFO sqlalchemy.engine.Engine [no key 0.00090s] ()
2022-10-10 08:41:28,777 INFO sqlalchemy.engine.Engine 
C

Above, the `User` class now has an _attribute_ `User.addresses` and the `Address` class has an _attribute_ `Address.user`. The `relationship()` construct will be used to __inspect the table relationships__ between the `Table` objects that are __mapped__ to the `User` and `Address` classes. As the `Table` object representing the _address table_ has a `ForeignKeyConstraint` which refers to the *user_account table*, the `relationship()` can _determine_ __unambiguously__ that there is a __one to many relationship__ _from_ `User.addresses` _to_ `User`; _one particular row_ in the `user_account` table __may be referred towards__ by _many rows_ in the `address` table.

_All one-to-many relationships_ __naturally correspond__ to a _many to one relationship_ in the _other direction_, in this case the one noted by `Address.user`. The `relationship.back_populates` parameter, seen above configured on both `relationship()` objects __referring__ to the _other name_, establishes that each of these two `relationship()` constructs should be __considered to be complimentary__ to _each other_; we will see how this plays out in the next section.

#### Persisting and Loading Relationships

We can start by _illustrating_ what `relationship()` does to instances of objects. If we make a new `User` object, we can note that there is a _Python list_ when we access the `.addresses` element.

In [6]:
u1 = User(name="pkrabs", fullname="Pearl Krabs")
print(f"{u1.addresses = }")

u1.addresses = []


This object is a _SQLAlchemy-specific version_ of _Python list_ which has the __ability to track and respond to changes__ made to it. The _collection_ also appeared __automatically__ when we _accessed the attribute_, even though we __never assigned__ it to the object. This is similar to the behavior noted at `Inserting Rows with the ORM` where it was observed that _column-based attributes_ to which we _don't explicitly assign a value_ also display as `None` __automatically__, rather than raising an `AttributeError` as would be Python's usual behavior.

As the `u1` object is still __transient__ and the _list_ that we got from `u1.addresses` has __not been mutated__ (i.e. _appended or extended_), it's __not actually `associated` with the object__ yet, but as we _make changes_ to it, it will become _part of the state_ of the `User` object.

The _collection_ is specific to the `Address` class which is the _only type of Python object_ that may be __persisted within it__. Using the `list.append()` method we may __add an Address object__.

In [7]:
a1 = Address(email_address="pearl.krabs@gmail.com")
u1.addresses.append(a1)

At this point, the `u1.addresses` _collection_ as expected _contains_ the __new `Address` object__.

In [8]:
print(f"{u1.addresses = }")

u1.addresses = [Address(id=None, email_address='pearl.krabs@gmail.com')]


As we _associated_ the `Address` object with the `User.addresses` _collection_ of the `u1` instance, another behavior also occurred, which is that the `User.addresses` _relationship_ __synchronized itself__ with the `Address.user` _relationship_, such that we __can navigate__ not only from the _User object to the Address object_, we __can also navigate__ from the _Address object back to the "parent" User object_.

In [9]:
print(f"{a1.user = }")

a1.user = User(id=None, name='pkrabs', fullname='Pearl Krabs')


This _synchronization_ occurred as a result of our _use of the_ `relationship.back_populates` parameter between the two `relationship()` objects. This parameter __names__ _another_ `relationship()` for which _complementary attribute_ __assignment/list mutation__ should occur. It will __work equally well__ in the _other direction_, which is that if we _create another_ `Address` _object_ and assign to its `Address.user` attribute, that `Address` becomes part of the `User.addresses` _collection_ on that `User` object.

In [10]:
a2 = Address(email_address="pearl@aol.com", user=u1)
print(f"{u1.addresses = }")

u1.addresses = [Address(id=None, email_address='pearl.krabs@gmail.com'), Address(id=None, email_address='pearl@aol.com')]


We actually made use of the `user` parameter as a _keyword argument_ in the `Address` constructor, which is __accepted just like any other mapped attribute__ that was _declared_ on the `Address` class. It is __equivalent__ to _assignment_ of the `Address.user` attribute after the fact.

In [11]:
# equivalent effect as a2 = Address(user=u1)
a2.user = u1

##### Cascading Objects into the Session

We now have a `User` and two `Address` objects that are _associated_ in a __bidirectional__ structure __in memory__, but as noted previously in `Inserting Rows with the ORM`, these objects are said to be in the __`transient state`__ until they are _associated_ with a `Session` object.

We make use of the `Session` that's __still ongoing__, and note that when we apply the `Session.add()` method to the lead `User` object, the _related_ `Address` object __also gets added__ to that _same_ `Session`.

In [12]:
session.add(u1)
print(f"{u1 in session = }")
print(f"{a1 in session = }")
print(f"{a2 in session = }")

u1 in session = True
a1 in session = True
a2 in session = True


The above behavior, where the `Session` _received_ a `User` object, and followed along the `User.addresses` _relationship_ to _locate a related_ `Address` object, is known as the __`save-update cascade`__.

The _three objects_ are now in the __pending state__; this means they are __ready__ to be the _subject_ of an `INSERT` operation but this has __not yet proceeded__; all _three objects_ have __no `primary key` assigned__ yet, and in addition, the `a1` and `a2` objects have an _attribute_ called `user_id` which refers to the `Column` that has a `ForeignKeyConstraint` referring to the `user_account.id` column; these are also `None` as the _objects_ are __not yet associated__ with a real database row.

In [13]:
print(f"{u1.id = }")
print(f"{a1.user_id = }")

u1.id = None
a1.user_id = None


It's at this _stage_ that we can see the very _great utility_ that the `unit of work process` provides; recall in the section `INSERT` usually _generates_ the `"values"` clause __automatically__, rows were inserted into the `user_account` and `address` tables using _some elaborate syntaxes_ in order to __automatically associate__ the `address.user_id` columns with those of the `user_account` rows. Additionally, it was __necessary__ that we __emit__ `INSERT` for `user_account` rows __first__, _before those of_ `address`, since rows in `address` are __dependent__ on their _parent row_ in `user_account` for a value in their `user_id` column.

When using the `Session`, all this _tedium_ is handled for us and even the most _die-hard_ SQL purist can __benefit from automation__ of `INSERT`, `UPDATE` and `DELETE` statements. When we `Session.commit()` the transaction _all steps invoke_ in the __correct order__, and furthermore the __newly generated `primary key`__ of the `user_account` row is applied to the `address.user_id` column _appropriately_.

In [14]:
session.commit()

2022-10-10 08:41:29,531 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-10 08:41:29,533 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-10 08:41:29,534 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ('pkrabs', 'Pearl Krabs')
2022-10-10 08:41:29,537 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-10-10 08:41:29,538 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ('pearl.krabs@gmail.com', 1)
2022-10-10 08:41:29,540 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-10-10 08:41:29,541 INFO sqlalchemy.engine.Engine [cached since 0.003861s ago] ('pearl@aol.com', 1)
2022-10-10 08:41:29,542 INFO sqlalchemy.engine.Engine COMMIT


#### Loading Relationships

In the last step, we called `Session.commit()` which emitted a `COMMIT` for the _transaction_, and then per `Session.commit.expire_on_commit` _expired all objects_ so that they __refresh__ for the _next transaction_.

When we __next__ _access an attribute_ on these objects, we'll see the `SELECT` __emitted__ for the _primary attributes_ of the row, such as when we view the __newly generated `primary key`__ for the `u1` object.

In [15]:
print(f"{u1.id = }")

2022-10-10 08:41:29,632 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-10 08:41:29,636 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-10-10 08:41:29,637 INFO sqlalchemy.engine.Engine [generated in 0.00153s] (1,)
u1.id = 1


The _u1_ `User` object now has a _persistent collection_ `User.addresses` that we may also access. As this _collection_ consists of an _additional set of rows_ from the `address` table, when we _access this collection_ as well we again see a __lazy load emitted__ in order to __retrieve the objects__.

In [16]:
print(f"{u1.addresses = }")

2022-10-10 08:41:29,723 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-10-10 08:41:29,725 INFO sqlalchemy.engine.Engine [generated in 0.00139s] (1,)
u1.addresses = [Address(id=1, email_address='pearl.krabs@gmail.com'), Address(id=2, email_address='pearl@aol.com')]


_Collections_ and _related attributes_ in the SQLAlchemy ORM are __persistent in memory__; once the _collection_ or _attribute_ is `populated`, _SQL_ is __no longer emitted__ until that _collection_ or _attribute_ is `expired`. We __may access__ `u1.addresses` again as well as _add_ or _remove_ items and this __will not incur__ any new SQL calls.

In [17]:
print(f"{u1.addresses = }")

u1.addresses = [Address(id=1, email_address='pearl.krabs@gmail.com'), Address(id=2, email_address='pearl@aol.com')]


While the loading _emitted_ by __`lazy loading`__ can _quickly become expensive_ if we __don't take explicit steps__ to _optimize_ it, the _network of lazy loading_ at least is __fairly well optimized__ to __not perform redundant work__; as the `u1.addresses` collection was __refreshed__, per the _identity map_ these are in fact the __same `Address` instances__ as the `a1` and `a2` objects we've been dealing with already, so we're _done loading all attributes_ in this particular __object graph__.

In [18]:
print(f"{a1 = }")
print(f"{a2 = }")

a1 = Address(id=1, email_address='pearl.krabs@gmail.com')
a2 = Address(id=2, email_address='pearl@aol.com')


The issue of how `relationships` load, or not, is an entire subject onto itself. Some additional introduction to these concepts is later in this section at `Loader Strategies`.

#### Using Relationships in Queries

The previous section introduced the _behavior_ of the `relationship()` construct when working with _instances of a mapped class_, above, the `u1`, `a1` and `a2` instances of the `User` and `Address` classes. In this section, we introduce the _behavior_ of `relationship()` as it applies to _class level behavior of a mapped class_, where it _serves_ in several ways to help __automate the construction of SQL queries__.

##### Using Relationships to Join

The sections `Explicit FROM clauses and JOINs` and `Setting the ON Clause` introduced the usage of the `Select.join()` and `Select.join_from()` methods to __compose__ _SQL JOIN clauses_. In order to describe __how to join between tables__, these methods either __infer__ the `ON clause` based on the presence of a _single unambiguous_ `ForeignKeyConstraint` object within the _table metadata structure_ that links the two tables, or otherwise we may _provide_ an __explicit__ `SQL Expression construct` that __indicates a specific ON clause__.

When _using ORM entities_, an _additional mechanism_ is available to help us set up the `ON clause` of a _join_, which is to make use of the `relationship()` objects that we set up in our user _mapping_, as was demonstrated at `Declaring Mapped Classes`. The _class-bound attribute_ corresponding to the `relationship()` may be passed as the __single argument__ to `Select.join()`, where it serves to __indicate both the `right side of the join` as well as the `ON clause` `at once`__.

In [19]:
print(select(Address.email_address).select_from(User).join(User.addresses))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


The presence of an ORM `relationship()` on a _mapping_ is __not used__ by `Select.join()` or `Select.join_from()` _if we don't specify it_; it is __not used__ for `ON clause` _inference_. This means, if we _join from User to Address without an ON clause_, it __works__ because of the `ForeignKeyConstraint` _between the two mapped_ `Table` _objects_, __not because__ of the `relationship()` objects on the `User` and `Address` classes.

In [20]:
print(select(Address.email_address).join_from(User, Address))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
